# LLM Infrastructure Demo

This notebook demonstrates the end-to-end pipeline using sample data for testing:
- Real-time document processing
- Audit logging
- Drift detection
- Explainability
- Compliance queries

**Note:** This notebook uses sample/test data for demonstration purposes. In production, all data comes from actual system operations.


In [1]:
import sys
import os
sys.path.append('../src')

from audit_logger import AuditLogger
from explainability import OutputValidator
from drift_detector import DriftDetector
import json
from datetime import datetime


Encryption not available. Install with: pip install cryptography


## 1. Initialize Components


In [ ]:
# Initialize audit logger with privacy features
audit_logger = AuditLogger(
    mask_sensitive_fields=True,
    anonymize_user_ids=False,
    encrypt_db=False
)

print("[OK] Audit logger initialized")


✅ Audit logger initialized


## 2. Process Sample Document & Log to Audit Trail


In [ ]:
# Sample financial document (for testing/demonstration only)
# In production, documents come from actual Kafka messages
sample_doc = """
Apple Inc. Q4 2024 Earnings Report

Revenue: $89.5 billion, up 1% year-over-year
Net Income: $22.96 billion
EPS: $1.46 per share
iPhone revenue: $43.8 billion
Services revenue: $22.3 billion

Contact: investor@apple.com
Phone: 408-996-1010
"""

# Simulate LLM response (for testing only)
# In production, this comes from actual LLM inference
mock_response = {
    'choices': [{
        'text': 'Revenue: $89.5B, Net Income: $22.96B, EPS: $1.46'
    }],
    'usage': {'total_tokens': 150}
}

# Log to audit trail
# Note: processing_time_ms would come from actual measurement in production
request_id = audit_logger.log_request(
    input_text=sample_doc,
    model_response=mock_response,
    metadata={
        'model_version': 'llama2-7b',
        'processing_time_ms': 45.2,  # Sample value for demo
        'tenant_id': 'test-tenant',
        'user_id': 'test-user',
        'source': 'demo-notebook'
    }
)

print(f"[OK] Logged request: {request_id}")
print(f"\nNote: Sensitive fields (email, phone) are masked in audit logs")
print("Note: This is sample data for demonstration. Production uses real system data.")


✅ Logged request: b487e00b-5099-4549-9cbe-cc7fcec4c32b

Note: Sensitive fields (email, phone) are masked in audit logs


In [ ]:
# Query audit logs
# In production, use actual tenant_id from your system
results = audit_logger.query_logs({
    'limit': 10  # Query all recent logs
})

print(f"Found {len(results)} audit log entries")
if results:
    print(f"\nLatest entry:")
    print(json.dumps(results[0], indent=2, default=str))
else:
    print("No audit log entries found. Process some documents first.")


Found 1 audit log entries

Latest entry:
{
  "id": 1,
  "request_id": "b487e00b-5099-4549-9cbe-cc7fcec4c32b",
  "timestamp": "2025-11-19T23:40:34.823626Z",
  "input_hash": "f742a37629b3e1250bf52e5cba77213cd84f8c675f4c7e97c7d06ea01d8bed7c",
  "input_text": "\nApple Inc. Q4 2024 Earnings Report\n\nRevenue: $89.5 billion, up 1% year-over-year\nNet Income: $22.96 billion\nEPS: $1.46 per share\niPhone revenue: $43.8 billion\nServices revenue: $22.3 billion\n\nContact: [EMAIL_MASKED]\nPhone: [PHONE_MASKED]\n",
  "model_version": "llama2-7b",
  "model_parameters": {},
  "output_text": "Revenue: $89.5B, Net Income: $22.96B, EPS: $1.46",
  "confidence_scores": null,
  "explanation": null,
  "processing_time_ms": 45.2,
  "tokens_used": 150,
  "tenant_id": "apple-corp",
  "user_id": "analyst-123",
  "source": "earnings-report",
  "status": "success",
  "error_message": null,
  "created_at": "2025-11-19 23:40:34"
}


In [ ]:
# Output validation
validator = OutputValidator()
output_text = mock_response['choices'][0]['text']

validation_result = validator.validate(output_text, sample_doc)

print("Validation Result:")
print(json.dumps(validation_result, indent=2))

if validation_result['valid']:
    print("\n[OK] Output validation passed")
else:
    print(f"\n[FAIL] Output validation failed: {validation_result['errors']}")


Validation Result:
{
  "valid": true,
  "errors": [],
  "warnings": [
    "Output seems too short"
  ],
  "score": 0.8,
  "metrics": {
    "currencies_found": 3,
    "percentages_found": 0,
    "dates_found": 0,
    "keywords_found": 3,
    "output_length": 48
  }
}

✅ Output validation passed


In [ ]:
# Privacy features demo
test_text = "Contact: john.doe@example.com, Phone: 555-123-4567"
masked = audit_logger._mask_sensitive_data(test_text)

print("Original:", test_text)
print("Masked:", masked)
print("\n[OK] Privacy masking working correctly")


Original: Contact: john.doe@example.com, Phone: 555-123-4567
Masked: Contact: [EMAIL_MASKED], Phone: [PHONE_MASKED]

✅ Privacy masking working correctly
